In [1]:
import pandas as pd
import numpy as np
import re

# 1. Read Data

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.shape

(3000, 23)

In [4]:
test.shape

(4398, 22)

# 2. Data Cleaning

There seems to be way more test cases than training cases ideally we'd want it the other way around. Lets clean up these data sets a bit.

In [5]:
train.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


The following columns can be dropped from our two datasets: belongs_to_collection, homepage, imdb_id, original_title, overview, poster_path, status, tagline.

In [6]:
drop_cols = ['belongs_to_collection','homepage','imdb_id','original_title','overview','poster_path','status','tagline','Keywords','crew','production_companies','production_countries','spoken_languages']

In [7]:
for col in drop_cols:
    train.drop(col, axis=1, inplace = True)
    test.drop(col, axis=1, inplace = True)

In [8]:
train.shape

(3000, 10)

In [9]:
test.shape

(4398, 9)

Now lets check null values and clean up the columns individually

In [10]:
train.isnull().sum()

id                    0
budget                0
genres                7
original_language     0
popularity            0
release_date          0
runtime               2
title                 0
cast                 13
revenue               0
dtype: int64

In [11]:
test.isnull().sum()

id                    0
budget                0
genres               16
original_language     0
popularity            0
release_date          1
runtime               4
title                 3
cast                 13
dtype: int64

## 2.1 genres
The genres column contains a dictionary for each entry. Lets first clean up the column to only have a string of genres. The dictionary contains each genre the film is apart of, some films are apart of more than one genre.

In [12]:
#function that cleans up the genre data; essentially it just extracts the values from the value-key pairs
def clean_genre(df):
    clean_list = []
    all_genres = df.genres
    for genre in all_genres:
        if genre is not np.nan:
            names = re.findall("name': '(.*?)'}", genre)
            clean_list.append(names)
        if genre is np.nan:
            clean_list.append('NULL')
    return clean_list

In [13]:
clean_genrelist = clean_genre(train)

In [14]:
train.genres = clean_genrelist

In [15]:
train.genres.head(5)

0                            [Comedy]
1    [Comedy, Drama, Family, Romance]
2                             [Drama]
3                   [Thriller, Drama]
4                  [Action, Thriller]
Name: genres, dtype: object

Now do the same for test dataset

In [16]:
clean_genrelist = clean_genre(test)

In [17]:
test.genres = clean_genrelist

In [18]:
test.genres.head(5)

0    [Adventure, Animation, Family, Fantasy]
1                  [Horror, Science Fiction]
2                          [Comedy, Romance]
3                      [Drama, War, Mystery]
4                     [History, Documentary]
Name: genres, dtype: object

Now that we've cleaned up the columns, lets do something about the **NULL** values. We can drop the train NULL values since there are only 7/3000 of them:

In [19]:
for x in range (0,len(train)):
    if train.loc[x,'genres'] == 'NULL':
        train.drop(x, axis=0, inplace = True)

In [20]:
train.shape

(2993, 10)

The 7 movies have been dropped from the train dataset, now lets manually edit the test dataset null genres

In [21]:
for x in range (0,len(test)):
    if test.loc[x,'genres'] == 'NULL':
        print(test.loc[x,:])

id                                                                3074
budget                                                               0
genres                                                            NULL
original_language                                                   en
popularity                                                    0.004425
release_date                                                    5/9/86
runtime                                                             95
title                                                Dangerously Close
cast                 [{'cast_id': 2, 'character': 'Randy McDevitt',...
Name: 73, dtype: object
id                                                                3794
budget                                                         8000000
genres                                                            NULL
original_language                                                   en
popularity                                           

At Index **73** is the movie 'Dangerously Close', it was an Action Thriller

In [22]:
#so we dont get error messages when making the updates: https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
pd.options.mode.chained_assignment = None

In [23]:
mylist = list(('Action','Thriller'))

In [24]:
test.genres[73] = mylist

At Index **793** is the movie 'Table for Five', it was a Drama

In [25]:
#function to fill in NULL genres for test df
def fill_genre(genres, index):
    mylist = list((genres))
    test.genres[index] = mylist
    print(test.genres[index])
    

In [26]:
fill_genre(['Action'],793)

['Action']


At Index **910** is the movie 'Valentino', it was an adventure drama

In [27]:
fill_genre(['Adventure','Drama'],910)

['Adventure', 'Drama']


At Index **1221** is the movie 'Street Knight', it was an Action Adventure

In [28]:
fill_genre(['Action','Adventure'],1221)

['Action', 'Adventure']


At Index **1442** is the movie 'My Son', it was a Drama

In [29]:
fill_genre(['Drama'],1442)

['Drama']


At Index **1615** is the movie 'Nasha Russia', it was a Comedy

In [30]:
fill_genre(['Comedy'],1615)

['Comedy']


At Index **1964** is the movie 'Nasha Russia', it was a Crime Action film.

In [31]:
fill_genre(['Crime','Action'],1964)

['Crime', 'Action']


I cannot find the movie at index **2062**, so it will remain blank.

At Index **2118** is the movie 'Duniyadari', it was a Comedy Drama film.

In [32]:
fill_genre(['Comedy','Drama'],2118)

['Comedy', 'Drama']


At Index **2213** is the movie 'Praying with Lior', it was a Documentary film.

In [33]:
fill_genre(['Documentary'],2213)

['Documentary']


At Index **2251** is the movie 'Teddy Bears' Picnic', it was a Comedy film.

In [34]:
fill_genre(['Comedy'],2251)

['Comedy']


At Index **2519** is the movie 'Glukhar v kino', it was a Comedy film.

In [35]:
fill_genre(['Comedy'],2519)

['Comedy']


At Index **3449** is the movie 'Lucky Lady', it was a Drama Comedy film.


In [36]:
fill_genre(['Drama','Comedy'],3449)

['Drama', 'Comedy']


At Index **3485** is the movie 'Death of a Dynasty', it was a Comedy film.

In [37]:
fill_genre(['Comedy'],3485)

['Comedy']


At index **3564** is the movie 'Fahrenheit 9/11', it was a documentary

In [38]:
fill_genre(['Documentary'],3564)

['Documentary']


At index **3817** is the movie 'Miesten välisiä keskusteluja', it was a Drama

In [39]:
fill_genre(['Drama'],3817)

['Drama']


Check that only one NULL from indx **2062** remains:

In [40]:
for x in range (0,len(test)):
    if test.loc[x,'genres'] == 'NULL':
        print(test.loc[x,:])

id                                                                5063
budget                                                               0
genres                                                            NULL
original_language                                                   ro
popularity                                                     0.03856
release_date                                                  10/21/05
runtime                                                             95
title                                                               15
cast                 [{'cast_id': 1, 'character': 'Hilde', 'credit_...
Name: 2062, dtype: object


Lets give the movie at **2062** the mode genre

In [41]:
#get string version of genres
genre_strings = str(test.genres)

In [42]:
test.genres = genre_strings

In [43]:
mode = test.genres.mode()

In [44]:
mode

0    0              [Adventure, Animation, Family, ...
dtype: object

In [45]:
fill_genre(['Adventure', 'Animation','Family'],2062)

['Adventure', 'Animation', 'Family']


In [46]:
#check that no more NULL genres exist
for x in range (0,len(test)):
    if test.loc[x,'genres'] == 'NULL':
        print(test.loc[x,:])

In [47]:
test.isnull().sum()

id                    0
budget                0
genres                0
original_language     0
popularity            0
release_date          1
runtime               4
title                 3
cast                 13
dtype: int64

In [48]:
train.isnull().sum()

id                    0
budget                0
genres                0
original_language     0
popularity            0
release_date          0
runtime               2
title                 0
cast                 13
revenue               0
dtype: int64

## 2.2 Cast
Movies with missing Cast values will be replaced with **'NONE'**

In [49]:
train.cast.fillna('NONE', inplace=True)

In [50]:
test.cast.fillna('NONE', inplace=True)

In [51]:
train.isnull().sum()

id                   0
budget               0
genres               0
original_language    0
popularity           0
release_date         0
runtime              2
title                0
cast                 0
revenue              0
dtype: int64

In [52]:
test.isnull().sum()

id                   0
budget               0
genres               0
original_language    0
popularity           0
release_date         1
runtime              4
title                3
cast                 0
dtype: int64

## 2.3 runtime
We will manually fill in the missing runtime values for test and train if we can.

In [53]:
train[train.runtime.isnull()]

,id,budget,genres,original_language,popularity,release_date,runtime,title,cast,revenue
1335,1336,6000000,[Drama],ru,0.292296,10/29/07,NaN,Королёв,"[{'cast_id': 3, 'character': '–°–µ—Ä–≥–µ–π –ö–...",31000
2302,2303,0,[Comedy],de,0.002229,3/14/96,NaN,Happy Weekend,"[{'cast_id': 0, 'character': 'Joachim Krippo',...",65335


In [54]:
#information googled
train.runtime[1335] = 130
train.runtime[2302] = 90

In [55]:
test[test.runtime.isnull()]

,id,budget,genres,original_language,popularity,release_date,runtime,title,cast
243,3244,0,"0 [Adventure, Animation, Family, ...",es,0.000464,3/20/81,NaN,La caliente niña Julietta,"[{'cast_id': 7, 'character': 'Julietta Santigo..."
1489,4490,0,"0 [Adventure, Animation, Family, ...",es,0.661399,6/6/14,NaN,"Pancho, el perro millonario","[{'cast_id': 3, 'character': 'Alberto', 'credi..."
1632,4633,0,"0 [Adventure, Animation, Family, ...",es,0.281500,11/3/78,NaN,Nunca en horas de clase,"[{'cast_id': 6, 'character': 'Susy', 'credit_i..."
3817,6818,0,"0 [Adventure, Animation, Family, ...",fi,0.011427,1/4/13,NaN,Miesten välisiä keskusteluja,"[{'cast_id': 0, 'character': 'Kari Mairisaari'..."


In [56]:
#information googled
test.runtime[243] = 93
test.runtime[1489] = 91
test.runtime[1632] = 100
test.runtime[3817] = 116

In [57]:
test.isnull().sum()

id                   0
budget               0
genres               0
original_language    0
popularity           0
release_date         1
runtime              0
title                3
cast                 0
dtype: int64

In [58]:
train.isnull().sum()

id                   0
budget               0
genres               0
original_language    0
popularity           0
release_date         0
runtime              0
title                0
cast                 0
revenue              0
dtype: int64

## 2.3 Release Date
Theres just one missing release date from the test dataset which we can fill manually.

In [59]:
test[test.release_date.isnull()]

,id,budget,genres,original_language,popularity,release_date,runtime,title,cast
828,3829,0,"0 [Adventure, Animation, Family, ...",en,0.009057,NaN,90.0,"Jails, Hospitals & Hip-Hop",[]


In [60]:
#information googled
test.release_date[828] = '05/01/00'

In [61]:
train.isnull().sum()

id                   0
budget               0
genres               0
original_language    0
popularity           0
release_date         0
runtime              0
title                0
cast                 0
revenue              0
dtype: int64

In [62]:
test.isnull().sum()

id                   0
budget               0
genres               0
original_language    0
popularity           0
release_date         0
runtime              0
title                3
cast                 0
dtype: int64

## 2.4 Title
We can drop this column because we only needed it to google missing information, it wont help with modelling.

In [63]:
train.drop('title', axis=1, inplace = True)
test.drop('title', axis=1, inplace = True)

In [64]:
test.isnull().sum()

id                   0
budget               0
genres               0
original_language    0
popularity           0
release_date         0
runtime              0
cast                 0
dtype: int64

In [65]:
train.isnull().sum()

id                   0
budget               0
genres               0
original_language    0
popularity           0
release_date         0
runtime              0
cast                 0
revenue              0
dtype: int64

# 3. Feature Engineering
## 3.1 Oringinal Language 
Since a big moajority (86% of the data according to kaggle) has english as the orignal language, I will breakdown the original language to either ENGLISH or FOREIGN and then get dummy variables for the two. 

In [66]:
test.head()

,id,budget,genres,original_language,popularity,release_date,runtime,cast
0,3001,0,"0 [Adventure, Animation, Family, ...",ja,3.851534,7/14/07,90.0,"[{'cast_id': 3, 'character': 'Tonio', 'credit_..."
1,3002,88000,"0 [Adventure, Animation, Family, ...",en,3.559789,5/19/58,65.0,"[{'cast_id': 2, 'character': 'Nancy Fowler Arc..."
2,3003,0,"0 [Adventure, Animation, Family, ...",en,8.085194,5/23/97,100.0,"[{'cast_id': 11, 'character': 'Maggie', 'credi..."
3,3004,6800000,"0 [Adventure, Animation, Family, ...",fr,8.596012,9/4/10,130.0,"[{'cast_id': 6, 'character': 'Nawal', 'credit_..."
4,3005,2000000,"0 [Adventure, Animation, Family, ...",en,3.217680,2/11/05,92.0,"[{'cast_id': 1, 'character': 'Narrator (voice)..."


In [67]:
train.head()

,id,budget,genres,original_language,popularity,release_date,runtime,cast,revenue
0,1,14000000,[Comedy],en,6.575393,2/20/15,93.0,"[{'cast_id': 4, 'character': 'Lou', 'credit_id...",12314651
1,2,40000000,"[Comedy, Drama, Family, Romance]",en,8.248895,8/6/04,113.0,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...",95149435
2,3,3300000,[Drama],en,64.299990,10/10/14,105.0,"[{'cast_id': 5, 'character': 'Andrew Neimann',...",13092000
3,4,1200000,"[Thriller, Drama]",hi,3.174936,3/9/12,122.0,"[{'cast_id': 1, 'character': 'Vidya Bagchi', '...",16000000
4,5,0,"[Action, Thriller]",ko,1.148070,2/5/09,118.0,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...",3923970


In [88]:
#replace all foreign languages with FOREIGN
for x in range (0,len(train)):
    if x in list(train.index):
        if train.loc[x,'original_language'] != 'en':
            train.loc[x,'original_language'] = 'FOREIGN'
    

In [97]:
for x in range (0,len(test)):
        if test.loc[x,'original_language'] != 'en':
            test.loc[x,'original_language'] = 'FOREIGN'

In [101]:
train = pd.get_dummies(train, columns=['original_language'])

In [103]:
test = pd.get_dummies(test, columns=['original_language'])

In [107]:
train.drop(['original_language'],axis = 1)
test.drop(['original_language'],axis = 1)

In [109]:
test.head(3)

,id,budget,genres,popularity,release_date,runtime,cast,original_language_FOREIGN,original_language_en
0,3001,0,"0 [Adventure, Animation, Family, ...",3.851534,7/14/07,90.0,"[{'cast_id': 3, 'character': 'Tonio', 'credit_...",1,0
1,3002,88000,"0 [Adventure, Animation, Family, ...",3.559789,5/19/58,65.0,"[{'cast_id': 2, 'character': 'Nancy Fowler Arc...",0,1
2,3003,0,"0 [Adventure, Animation, Family, ...",8.085194,5/23/97,100.0,"[{'cast_id': 11, 'character': 'Maggie', 'credi...",0,1


In [110]:
train.head(3)

,id,budget,genres,popularity,release_date,runtime,cast,revenue,original_language_FOREIGN,original_language_en
0,1,14000000,[Comedy],6.575393,2/20/15,93.0,"[{'cast_id': 4, 'character': 'Lou', 'credit_id...",12314651,0,1
1,2,40000000,"[Comedy, Drama, Family, Romance]",8.248895,8/6/04,113.0,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...",95149435,0,1
2,3,3300000,[Drama],64.299990,10/10/14,105.0,"[{'cast_id': 5, 'character': 'Andrew Neimann',...",13092000,0,1


## 3.2 Release Date
I will create two columns from the release date - one for year released and one for month released. I will then derive the northern hempisphere season (winter, summer, autumn or spring) from the months.

In [120]:
dates = list(train.release_date)

In [126]:
#extract years
years = list()
for date in dates:
    years.append(re.findall('..$',date))

In [128]:
#add years to train df
train['years']=years

In [129]:
train.head()

,id,budget,genres,popularity,release_date,runtime,cast,revenue,original_language_FOREIGN,original_language_en,years
0,1,14000000,[Comedy],6.575393,2/20/15,93.0,"[{'cast_id': 4, 'character': 'Lou', 'credit_id...",12314651,0,1,[15]
1,2,40000000,"[Comedy, Drama, Family, Romance]",8.248895,8/6/04,113.0,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...",95149435,0,1,[04]
2,3,3300000,[Drama],64.299990,10/10/14,105.0,"[{'cast_id': 5, 'character': 'Andrew Neimann',...",13092000,0,1,[14]
3,4,1200000,"[Thriller, Drama]",3.174936,3/9/12,122.0,"[{'cast_id': 1, 'character': 'Vidya Bagchi', '...",16000000,1,0,[12]
4,5,0,"[Action, Thriller]",1.148070,2/5/09,118.0,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...",3923970,1,0,[09]


In [130]:
#do the same for test
dates = list(test.release_date)
#extract years
years = list()
for date in dates:
    years.append(re.findall('..$',date))
#add years to train df
test['years']=years
test.head()

,id,budget,genres,popularity,release_date,runtime,cast,original_language_FOREIGN,original_language_en,years
0,3001,0,"0 [Adventure, Animation, Family, ...",3.851534,7/14/07,90.0,"[{'cast_id': 3, 'character': 'Tonio', 'credit_...",1,0,[07]
1,3002,88000,"0 [Adventure, Animation, Family, ...",3.559789,5/19/58,65.0,"[{'cast_id': 2, 'character': 'Nancy Fowler Arc...",0,1,[58]
2,3003,0,"0 [Adventure, Animation, Family, ...",8.085194,5/23/97,100.0,"[{'cast_id': 11, 'character': 'Maggie', 'credi...",0,1,[97]
3,3004,6800000,"0 [Adventure, Animation, Family, ...",8.596012,9/4/10,130.0,"[{'cast_id': 6, 'character': 'Nawal', 'credit_...",1,0,[10]
4,3005,2000000,"0 [Adventure, Animation, Family, ...",3.217680,2/11/05,92.0,"[{'cast_id': 1, 'character': 'Narrator (voice)...",0,1,[05]
